# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import sys
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)
from sqlalchemy import create_engine

# For machine learning
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, make_scorer,classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

import warnings

warnings.simplefilter('ignore')

# For nlp
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline, FeatureUnion
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
import pickle

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/njackson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/njackson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/njackson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///data/data.db')
df = pd.read_sql_table(table_name='disaster_response', con=engine)
X = df['message']
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
from nltk.corpus import stopwords
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'

def tokenize(text):
    # Detect URLs
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
    
    # Normalize and tokenize and remove punctuation
    tokens = nltk.word_tokenize(re.sub(r"[^a-zA-Z0-9]", " ", text.lower()))
    
    # Remove stopwords
    tokens = [t for t in tokens if t not in stopwords.words('english')]

    # Lemmatize
    lemmatizer=WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    
    return tokens

In [4]:
# test the tokenizer 
test = ' Here is some text that I believe will be a good test for my Tokenizer that was Written in Virginia, and not DC~!'

In [5]:
tokenize(test)

['text', 'believe', 'good', 'test', 'tokenizer', 'written', 'virginia', 'dc']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer = tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))])

In [7]:
pipeline.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'vect', 'tfidf', 'clf', 'vect__analyzer', 'vect__binary', 'vect__decode_error', 'vect__dtype', 'vect__encoding', 'vect__input', 'vect__lowercase', 'vect__max_df', 'vect__max_features', 'vect__min_df', 'vect__ngram_range', 'vect__preprocessor', 'vect__stop_words', 'vect__strip_accents', 'vect__token_pattern', 'vect__tokenizer', 'vect__vocabulary', 'tfidf__norm', 'tfidf__smooth_idf', 'tfidf__sublinear_tf', 'tfidf__use_idf', 'clf__estimator__bootstrap', 'clf__estimator__ccp_alpha', 'clf__estimator__class_weight', 'clf__estimator__criterion', 'clf__estimator__max_depth', 'clf__estimator__max_features', 'clf__estimator__max_leaf_nodes', 'clf__estimator__max_samples', 'clf__estimator__min_impurity_decrease', 'clf__estimator__min_impurity_split', 'clf__estimator__min_samples_leaf', 'clf__estimator__min_samples_split', 'clf__estimator__min_weight_fraction_leaf', 'clf__estimator__n_estimators', 'clf__estimator__n_jobs', 'clf__estimator__oob_score', 'clf_

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y,random_state = 42)

In [9]:
%%time
pipeline.fit(X_train, Y_train)

CPU times: user 8min 10s, sys: 26.3 s, total: 8min 37s
Wall time: 9min 36s


Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f8f8bed66a8>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [10]:
Y_pred = pipeline.predict(X_test)
    
# Calculate the accuracy for each of them.
for i in range(len(Y.columns)):
    print('Category: {} '.format(Y.columns[i]))
    print(classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i])))
    print('F1 {}\n\n'.format(f1_score(Y_test.iloc[:, i].values, Y_pred[:, i],average='weighted')))

Category: related 
              precision    recall  f1-score   support

           0       0.69      0.39      0.50      1563
           1       0.83      0.94      0.88      4944
           2       0.35      0.34      0.34        47

    accuracy                           0.81      6554
   macro avg       0.62      0.56      0.58      6554
weighted avg       0.79      0.81      0.79      6554

Accuracy 0.8053097345132744


F1 0.7859711688589983


Category: request 
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5443
           1       0.84      0.49      0.62      1111

    accuracy                           0.90      6554
   macro avg       0.87      0.74      0.78      6554
weighted avg       0.89      0.90      0.89      6554

Accuracy 0.8979249313396399


F1 0.8865083397035343


Category: offer 
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6521
           1       0

### 6. Improve your model
Use grid search to find better parameters. 

In [16]:
%%time
#grid search

parameters = {'clf__estimator__n_estimators':[10, 20], 
              'clf__estimator__min_samples_split':[2, 4, 8]}

pipeline_cv = GridSearchCV(estimator=pipeline, param_grid=parameters, verbose=3)

CPU times: user 5.89 ms, sys: 36.3 ms, total: 42.2 ms
Wall time: 43.7 ms


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [17]:
%%time
pipeline_cv.fit(X_train, Y_train)
print('Best Parameters:', pipeline_cv.best_params_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.245, total= 2.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.3min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.236, total= 2.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 


[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:  4.7min remaining:    0.0s


[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.240, total= 2.4min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.229, total= 2.3min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=10, score=0.248, total= 2.2min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.249, total= 2.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.253, total= 2.9min
[CV] clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20 
[CV]  clf__estimator__min_samples_split=2, clf__estimator__n_estimators=20, score=0.262, total= 3.0min
[CV] cl

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed: 79.3min finished


Best Parameters: {'clf__estimator__min_samples_split': 2, 'clf__estimator__n_estimators': 20}
CPU times: user 1h 5min 21s, sys: 11min 5s, total: 1h 16min 26s
Wall time: 1h 23min 10s


In [18]:
pickle.dump(pipeline_cv, open('disaster_model.sav', 'wb'))

In [19]:
loaded_model = pickle.load(open('disaster_model.sav', 'rb'))

In [33]:
X_test.shape

(6554,)

In [27]:
Y_pred = loaded_model.predict(X_test)
    
print(classification_report(Y_test.iloc[:, 1:].values, np.array([x[1:] for x in Y_pred])))

              precision    recall  f1-score   support

           0       0.81      0.48      0.60      1111
           1       0.00      0.00      0.00        33
           2       0.73      0.66      0.69      2670
           3       0.61      0.11      0.19       535
           4       0.75      0.10      0.18       344
           5       0.61      0.07      0.12       159
           6       0.25      0.01      0.02       116
           7       0.58      0.07      0.13       200
           8       0.00      0.00      0.00         0
           9       0.87      0.35      0.50       418
          10       0.83      0.59      0.69       745
          11       0.81      0.35      0.49       581
          12       0.75      0.15      0.25        98
          13       0.78      0.05      0.10       133
          14       0.00      0.00      0.00        73
          15       0.50      0.01      0.02       215
          16       0.83      0.16      0.28       297
          17       0.59    

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
new_pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(
        AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))
    ))
])
nltk.download('averaged_perceptron_tagger')
new_pipeline.fit(X_train, Y_train)


In [ ]:
Y_pred = new_pipeline.predict(X_test)
    
# Calculate the accuracy for each of them.
for i in range(len(Y.columns)):
    print('Category: {} '.format(Y.columns[i]))
    print(classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy {}\n\n'.format(accuracy_score(Y_test.iloc[:, i].values, Y_pred[:, i])))
    print('F1 {}\n\n'.format(f1_score(Y_test.iloc[:, i].values, Y_pred[:, i],average='weighted')))

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.